In [1]:
import networkx as nx
import pandas as pd

from Bio import SeqIO
from Levenshtein import distance

# Introduction

There are particular polymorphisms that are found that have been experimentally shown (on a limited scale) to confer host adaptation mutations. I want to see whether there is evidence of these polymorphisms flowing into human populations, or if they instead arise independently post-infection.

12 March 2015: For simplicity, I am first starting with the PB2 gene only, and looking only at E627K.

In [4]:
# Open the sequences
sequences = SeqIO.to_dict(SeqIO.parse('20150312_PB2_CDS_from_Whole_Genomes.fasta', 'fasta'))
# sequences

In [30]:
data = pd.read_csv('20150312_PB2_CDS_from_Whole_Genomes.csv', parse_dates=['Collection Date'])
data['Strain Name'] = data['Strain Name'].str.split('(').str[0]
data['Sequence Accession'] = data['Sequence Accession'].str.strip('*')
data
accession_strain = dict(zip(data['Sequence Accession'], data['Strain Name']))
accession_strain

{'AB049153': 'A/parakeet/Chiba/1/97',
 'AB049154': 'A/parakeet/Narita/92A/98',
 'AB166859': 'A/chicken/Yamaguchi/7/2004',
 'AB188813': 'A/chicken/Oita/8/2004',
 'AB188821': 'A/chicken/Kyoto/3/2004',
 'AB189050': 'A/crow/Kyoto/53/2004',
 'AB189058': 'A/crow/Osaka/102/2004',
 'AB212050': 'A/Hong Kong/213/2003',
 'AB212051': 'A/Hong Kong/213/2003',
 'AB212277': 'A/duck/Yokohama/aq10/2003',
 'AB239300': 'A/bar-headed goose/Mongolia/1/05',
 'AB239307': 'A/whooper swan/Mongolia/3/05',
 'AB253760': 'A/R',
 'AB256663': 'A/chicken/Osaka/aq48/97',
 'AB256671': 'A/chicken/Yokohama/aq55/2001',
 'AB256679': 'A/chicken/Kobe/aq26/2001',
 'AB256687': 'A/chicken/Osaka/aq58/2001',
 'AB256695': 'A/chicken/Osaka/aq69/2001',
 'AB256703': 'A/chicken/Yokohama/aq45/2002',
 'AB256711': 'A/chicken/Yokohama/aq134/2002',
 'AB256719': 'A/chicken/Yokohama/aq135/2001',
 'AB256727': 'A/chicken/Yokohama/aq120/2001',
 'AB256735': 'A/chicken/Osaka/aq19/2001',
 'AB256743': 'A/chicken/Yokohama/aq144/2001',
 'AB259709': 'A

In [31]:
G = nx.read_gpickle('20141103 All IRD Final Graph.pkl')
G.nodes()

['A/mallard/Interior Alaska/6MP0160AR1/2006',
 'A/Memphis/32/1983',
 'A/swine/Hong Kong/NS970/2012',
 'A/guinea fowl/New Jersey/7290-19/1994',
 'A/California/VRDL88/2009',
 'A/tufted duck/PT/13771/2006',
 'A/Boston/YGA_00093/2012',
 'A/village chicken/Kyaing Tong/2433/2007',
 'A/Peru/PER383/2010',
 'A/baikal teal/Korea/Donglim3/2014',
 'A/northern pintail/Interior Alaska/9BM6248R0/2009',
 'A/greater white-fronted goose/California/44358-089/2007',
 'A/swine/Ohio/10SW126/2010',
 'A/Duck/Nanchang/2-0485/2000',
 'A/Houston/1H/2009',
 'A/Texas/44301765/2009',
 'A/Canada-AB/RV1532/2009',
 'A/Texas/45071524/2009',
 'A/swine/Minnesota/SG1236/2005',
 'A/mallard/Wisconsin/986/1982',
 'A/California/NHRC0008/2005',
 'A/Memphis/9/1980',
 'A/mallard/Sweden/80114/2008',
 'A/Helsinki/1229/2013',
 'A/Canterbury/05/2001',
 'A/Canterbury/05/2002',
 'A/Malaysia/2143035/2009',
 'A/Boston/YGA_00035/2013',
 'A/swine/Guangdong/01/2008',
 'A/Tennessee/F2048A/2011',
 'A/mallard/Sweden/68583/2007',
 'A/Florida/U

We need a function to figure out what polymorphism exists in a particular position along the PB2 gene. However, this is complicated by the fact that there are PB2 sequences with indels, hence the exact numbering might not be the same. To get around this, I will take a "best match" approach that Justin Z. coded up earlier on. To reduce the search space, I will also tokenize a short region ±40 a.a.

In [84]:
def tokenize_string(string, token_length, start_pos, end_pos):
    """
    Takes an input string, and returns a dictionary where:
        - key = position, and 
        - value = a tokenized string of length token_length
    """
    assert end_pos > start_pos, "End position must be greater than start position."
    assert (end_pos - start_pos) > token_length, "Token length must be smaller than the position range."
    
    tokens = dict()
    
    for i in range(start_pos, end_pos-token_length):
        tokens[string[i:i+token_length]] = i
        
        
    return tokens

# tokenize_string(eg_seq, 20, 627-30, 627+30)

# Of the tokenized strings, there will be a few with a "best match" i.e. minimum distance. 
# We are only interested in comparing against that one.
def get_best_tokens(tokens, query):
    """
    Takes in:
    - a dictionary containing the tokens of a string & their position.
    - a query string of equal length to the keys in the dictionary.
    
    Returns:
    - a list of tokens that have the smallest Levenshtein distance to the query string.
    """
    from Levenshtein import distance
    
    best_tokens = set()
    min_distance = len(query)
    for token, position in tokens.items():
        if distance(token, query) < min_distance:
            min_distance = distance(token, query)
            best_tokens = set()
            best_tokens.add(token)
        if distance(token, query) == min_distance:
            best_tokens.add(token)
        if distance(token, query) > min_distance:
            pass
    
    return(list(best_tokens))

In [86]:
# Let's then setup a reference string that contains a E at the 627th position in the a.a. sequence.
# - The first letter in the reference string will be the letter E.
# - The length of the string will be 20 a.a. long.
# Looking to the future, we call it a match if at least 17 out of 20 a.a. are identical.

eg_seq = str(sequences['CY005153'].seq)
refstring = eg_seq[626:626+20]

idx = 6720
for i, (accession, seqrecord) in enumerate(sequences.items()):
    tokens = tokenize_string(str(seqrecord.seq), 20, 627-20, 627+20)
    not_present = set()
    best_tokens = get_best_tokens(tokens, refstring)
    strain_name = accession_strain[accession]
    if len(best_tokens) == 1 and strain_name in G.nodes():
        # Add in the polymorphism into the node.
        G.node[strain_name]['627aa'] = string[0]
    if len(best_tokens) > 1:
        print(strain_name, strain_name in G.nodes(), best_tokens)

('A/northern shoveler/Minnesota/Sg-00645/2008', False, ['DSQTATKRIRMAIN', 'TDSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00514/2008', False, ['DSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00528/2008', False, ['IRMAIN', 'QTATKRIRMAIN', 'RMAIN', 'KRIRMAIN', 'RIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00504/2008', False, ['DSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00554/2008', False, ['DSQTAT', 'SQTAT'])
('A/blue-winged teal/Minnesota/Sg-00649/2008', False, ['DSQTATKRIRMAIN', 'TDSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00537/2008', False, ['DSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/New Jersey/Sg-00516/2008', False, ['DSQTATKRIR', 'SQTATKRIR'])
('A/ruddy turnstone/New Jersey/Sg-00487/2008', False, ['DSQTATKRIRMAIN', 'SQTATKRIRMAIN'])
('A/ruddy turnstone/Delaware/Sg-00468/2008', False, ['DSQTATKRIR', 'SQTATKRIR'])
('A/ruddy turnstone/New Jersey/Sg-00508/2008', False, ['DSQTAT

In [68]:
refstring

'EQSRMQFSSLTVNVRGSGMR'

In [69]:
[k for k,v in accession_strain.items() if v == 'A/South Australia/36/2000']

['CY017154']

In [87]:
# Which nodes don't have the '627aa' metadata?
[n for n, d in G.nodes(data=True) if '627aa' not in d.keys()]

['A/mallard/Wisconsin/2712/2009',
 'A/mallard/Alberta/11/91',
 'A/Phillipines/2/1982',
 'A/HaNoi/Q177/2006',
 'A/Swine/Fujian/F1/2001',
 'A/mallard/Washington/44338-034/2007',
 'A/Nymc X-157-A/St.Petersburg/8/2006)_RX',
 'A/duck/Interior Alaska/7MP1598/2007',
 'A/swine/Binh Duong/03_10/2010',
 'A/chicken/INDIA/NIV33491/06',
 'A/mallard duck/ALB/635/1983',
 'A/duck/Nanchang/2-0485/2000',
 'A/guinea fowl/New York/32084/2006',
 'A/HaNoi/HN30109/2004',
 'A/env/California/7262/2008',
 'A/quail/Hong Kong/SF550/00 ',
 'A/mallard/Washington/44338-049/2007',
 'A/mallard/Washington/44242-124/2006',
 'A/mallard duck/ALB/884/1984',
 'A/mallard/Interior Alaska/9BM2073/2009',
 'A/pintail duck/ALB/189/1982',
 'A/env/California/7341/2008',
 'A/mallard/Washington/44242-144/2006',
 'A/Santo Domingo/WRAIR3516T/2010',
 'A/laughing gull/DE/2838/1987',
 'A/mallard/Wisconsin/2756/2009',
 'A/mallard/Minnesota/SG-00932/2008',
 'A/mallard/Sweden/55/2002',
 'A/swine/Yunnan/74/2009',
 'A/Swine/Spain/SF32071/2007'

In [70]:
tokens = tokenize_string(str(sequences['CY017154'].seq), 20, 627-20, 627+20, )
print(tokens)
for k, v in tokens.items():
    print(distance(refstring, k))

{'DTIQIIKLLPFAAAPPKQSR': 610, 'APPKQSRMQFSSLTVNVRGA': 623, 'IQIIKLLPFAAAPPKQSRMQ': 612, 'PKQSRMQFSSLTVNVRGAGM': 625, 'KLLPFAAAPPKQSRMQFSSL': 616, 'IIKLLPFAAAPPKQSRMQFS': 614, 'TFDTIQIIKLLPFAAAPPKQ': 608, 'KQSRMQFSSLTVNVRGAGMR': 626, 'PFAAAPPKQSRMQFSSLTVN': 619, 'LLPFAAAPPKQSRMQFSSLT': 617, 'GTFDTIQIIKLLPFAAAPPK': 607, 'PPKQSRMQFSSLTVNVRGAG': 624, 'LPFAAAPPKQSRMQFSSLTV': 618, 'IKLLPFAAAPPKQSRMQFSS': 615, 'TIQIIKLLPFAAAPPKQSRM': 611, 'QIIKLLPFAAAPPKQSRMQF': 613, 'AAAPPKQSRMQFSSLTVNVR': 621, 'AAPPKQSRMQFSSLTVNVRG': 622, 'FDTIQIIKLLPFAAAPPKQS': 609, 'FAAAPPKQSRMQFSSLTVNV': 620}
19
8
17
4
20
19
18
2
15
19
19
6
17
19
18
18
11
9
19
13


In [72]:
from collections import Counter
Counter([d['host_species'] for n, d in G.nodes(data=True) if '627aa' in d.keys() and d['627aa'] == 'K'])

Counter({'Human': 4689, 'Swine': 117, 'Chicken/Avian': 98, 'Duck/Avian': 17, 'Mute Swan/Avian': 13, 'Turkey/Avian': 9, 'Avian': 7, 'Ferret': 6, 'Goose/Avian': 6, 'Whooper Swan/Avian': 4, 'Ostrich/Avian': 3, 'Pigeon/Avian': 3, 'Grebe/Avian': 3, 'Bar-Headed Goose/Avian': 2, 'Laughing Gull/Avian': 2, 'Swan/Avian': 2, 'Falcon/Avian': 2, 'Tree Sparrow/Avian': 2, 'Unknown': 1, 'Openbill Stork/Avian': 1, 'Rook/Avian': 1, 'Rhea/Avian': 1, 'Saker Falcon/Avian': 1, 'Green-Winged Teal/Avian': 1, 'Northern Pintail/Avian': 1, 'Starling/Avian': 1, 'Magpie/Avian': 1, 'Muscovy Duck/Avian': 1, 'Common Goldeneye/Avian': 1, 'Guineafowl/Avian': 1, 'Weasel': 1, 'Eagle/Avian': 1})